In [29]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.datasets import DatasetFolder
from PIL import Image
import torch.nn as nn

In [30]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.Resize((28, 28)),
])

train_dataset = datasets.ImageFolder(root="archive/train", transform=transform, )
test_dataset = datasets.ImageFolder(root='archive/test', transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32)


In [31]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding = 1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding = 1)
        self.pool = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128,10)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = SimpleCNN()
print(model)



SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1568, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [32]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [33]:
number_of_epochs = 5

for epoch in range(number_of_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        print(f"Image batch shape: {images.size()}")
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        print(f"Epoc {epoch+1}/{number_of_epochs}, Loss: {running_loss/len(train_loader):.4f}")


Image batch shape: torch.Size([32, 3, 28, 28])
Epoc 1/5, Loss: 0.0267
Image batch shape: torch.Size([32, 3, 28, 28])
Epoc 1/5, Loss: 0.0522
Image batch shape: torch.Size([32, 3, 28, 28])
Epoc 1/5, Loss: 0.0761
Image batch shape: torch.Size([32, 3, 28, 28])
Epoc 1/5, Loss: 0.0989
Image batch shape: torch.Size([32, 3, 28, 28])
Epoc 1/5, Loss: 0.1209
Image batch shape: torch.Size([32, 3, 28, 28])
Epoc 1/5, Loss: 0.1408
Image batch shape: torch.Size([32, 3, 28, 28])
Epoc 1/5, Loss: 0.1603
Image batch shape: torch.Size([32, 3, 28, 28])
Epoc 1/5, Loss: 0.1795
Image batch shape: torch.Size([32, 3, 28, 28])
Epoc 1/5, Loss: 0.1984
Image batch shape: torch.Size([32, 3, 28, 28])
Epoc 1/5, Loss: 0.2172
Image batch shape: torch.Size([32, 3, 28, 28])
Epoc 1/5, Loss: 0.2362
Image batch shape: torch.Size([32, 3, 28, 28])
Epoc 1/5, Loss: 0.2551
Image batch shape: torch.Size([32, 3, 28, 28])
Epoc 1/5, Loss: 0.2740
Image batch shape: torch.Size([32, 3, 28, 28])
Epoc 1/5, Loss: 0.2916
Image batch shape: t

In [34]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        print(images, "\n")
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print(f"Accuracy on test set: {100 * correct / total:.2f}%")

tensor([[[[-0.4516, -0.4284, -0.4182,  ..., -0.8319, -0.8032, -0.7460],
          [-0.4468, -0.4317, -0.4242,  ..., -0.8806, -0.8340, -0.7268],
          [-0.4453, -0.4431, -0.4325,  ..., -0.7636, -0.8252, -0.7136],
          ...,
          [ 0.7014,  0.8983,  0.9640,  ..., -0.1839, -0.5905, -0.5856],
          [ 0.8682,  0.8681,  0.9437,  ..., -0.5935, -0.6400, -0.5092],
          [ 0.7416,  0.8185,  0.9548,  ..., -0.6122, -0.6518, -0.5079]],

         [[ 0.1316,  0.1547,  0.1670,  ..., -0.4757, -0.4700, -0.3173],
          [ 0.1565,  0.1686,  0.1842,  ..., -0.5218, -0.4515, -0.4155],
          [ 0.1634,  0.1740,  0.1874,  ..., -0.2560, -0.4623, -0.4155],
          ...,
          [ 0.7874,  0.9358,  0.9759,  ...,  0.0107, -0.4350, -0.4094],
          [ 0.9213,  0.9199,  0.9654,  ..., -0.3919, -0.4812, -0.3418],
          [ 0.8142,  0.8732,  0.9671,  ..., -0.4389, -0.5039, -0.3644]],

         [[ 0.8663,  0.8967,  0.9103,  ..., -0.1453, -0.1687,  0.0908],
          [ 0.8794,  0.8986,  